In [7]:
import pandas as pd
import seaborn as sb
import numpy as np
import math as m
from matplotlib import pyplot as plt

np.set_printoptions(formatter={'float': lambda x: "{0:0.2f}".format(x)})

# Reader score must be set to a very small value otherwise there will be a division by 0

epsilon = 0.000001

# CSV file parsing

info = pd.read_csv("data/info.csv")
paper_authors = pd.read_csv("data/authors.csv")
paper_authors = paper_authors.values
paper_ratings = pd.read_csv("data/ratings.csv")
paper_ratings = paper_ratings.values

# Initial setup

dataset_name = info["Dataset"][0]
papers_number = info["Paper"][0]
readers_number = info["Reader"][0]
ratings_number = info["Rating"][0]
authors_number = info["Author"][0]
papers = np.arange(papers_number)
readers = np.arange(readers_number)
ratings = np.arange(ratings_number)
authors = np.arange(authors_number)
paper_steadiness = np.zeros(papers_number)
paper_score = np.zeros(papers_number)
rating_goodness = np.empty(ratings_number)
reader_steadiness = np.zeros(readers_number)
reader_score = np.zeros(readers_number)
reader_score.fill(epsilon)
author_steadiness = np.zeros(authors_number)
author_score = np.zeros(authors_number)


def get_author(current_paper):
    found_authors = []
    for author_index, author_entry in enumerate(paper_authors):
        current_author = int(author_entry[0])
        written_papers = author_entry[1].split(";")
        written_papers = [int(x) for x in written_papers]
        if current_paper in written_papers :
            found_authors.append(current_author)
    return np.asarray(found_authors)


for index, entry in enumerate(paper_ratings):

    # Example: <1,1,2,0.8,0>
    # At Timestamp 1 Reader 1 gave to Paper 2 a Rating of 0.8 written by Author 0
    timestamp = int(entry[0])
    reader = int(entry[1])
    paper = int(entry[2])
    rating = entry[3]
    authors_of_paper = get_author(paper)

    # COMPUTATION START: PAPER AND READER SCORE

    # Saving values at time t(i)

    old_paper_steadiness = paper_steadiness[paper]
    old_paper_score = paper_score[paper]
    old_reader_steadiness = reader_steadiness[reader]
    old_rating_goodness = rating_goodness[timestamp]
    old_reader_score = reader_score[reader]

    # Updating values at time t(i+1)

    paper_steadiness[paper] = old_paper_steadiness + old_reader_score
    paper_score[paper] = ((old_paper_steadiness * old_paper_score) + (old_reader_score * rating)) / paper_steadiness[paper]
    rating_goodness[timestamp] = 1 - (m.sqrt(abs(rating - paper_score[paper])))
    reader_steadiness[reader] = (old_reader_steadiness + paper_steadiness[paper])
    reader_score[reader] = ((old_reader_steadiness * old_reader_score) + (paper_steadiness[paper] * rating_goodness[timestamp])) / reader_steadiness[reader]

    # COMPUTATION START: AUTHOR SCORE

    for author in authors_of_paper:
        
        # Saving values at time t(i)

        old_author_steadiness = author_steadiness[author]
        old_author_score = author_score[author]

        # Updating values at time t(i+1)

        author_steadiness[author] = old_author_steadiness + old_reader_score
        author_score[author] = ((old_author_steadiness * old_author_score) + (old_reader_score * rating)) / author_steadiness[author]

    # COMPUTATION START: PROPAGATING CHANGES TO PREVIOUS READERS

    if index > 0:

        previous_paper_ratings = paper_ratings[:(index - 1)]

        for previous_index, previous_entry in enumerate(previous_paper_ratings):

            previous_timestamp = int(previous_entry[0])
            previous_reader = int(previous_entry[1])
            previous_paper = int(previous_entry[2])
            previous_rating = previous_entry[3]
            previous_author = int(previous_entry[4])

            if paper == previous_paper and reader != previous_reader:
                
                # Saving previous values at time t(i)

                old_previous_reader_steadiness = reader_steadiness[previous_reader]
                old_previous_reader_score = reader_score[previous_reader]
                old_previous_rating = previous_rating
                old_previous_rating_goodness = rating_goodness[previous_timestamp]

                # Updating previous values at time t(i+1)

                previous_rating_goodness = 1 - (m.sqrt(abs(previous_rating - paper_score[paper])))
                reader_steadiness[previous_reader] = (old_previous_reader_steadiness + old_reader_score)
                reader_score[previous_reader] = ((old_previous_reader_steadiness * old_previous_reader_score) +
                                                 (old_paper_steadiness * old_rating_goodness) +
                                                 (paper_steadiness[paper] * previous_rating_goodness)) \
                                                / reader_steadiness[previous_reader]


In [8]:
# Summary

print("PAPER SCORE:       ", paper_score)
print("PAPER STEADINESS:  ", paper_steadiness)
print("READER SCORE:      ", reader_score)
print("READER STEADINESS: ", reader_steadiness)
print("AUTHOR SCORE:      ", author_score)
print("AUTHOR STEADINESS: ", author_steadiness)

# Charts

print("CHART: PAPER / PAPER SCORE")
dataframe = pd.DataFrame({'Paper' : papers, 'Score' : paper_score})
dataframe.plot('Paper', 'Score', kind='bar')
plt.show()

print("CHART: PAPER / PAPER STEADINESS")
dataframe = pd.DataFrame({'Paper': papers, 'Steadiness': paper_steadiness})
dataframe.plot('Paper', 'Steadiness', kind='bar')
plt.show()

print("CHART: READER / READER SCORE")
dataframe = pd.DataFrame({'Reader': readers, 'Score': reader_score})
dataframe.plot('Reader', 'Score', kind='bar')
plt.show()

print("CHART: READER / READER STEADINESS")
dataframe = pd.DataFrame({'Reader': readers, 'Steadiness': reader_steadiness})
dataframe.plot('Reader', 'Steadiness', kind='bar')
plt.show()

print("CHART: AUTHOR / AUTHOR SCORE")
dataframe = pd.DataFrame({'Author': authors, 'Score': author_score})
dataframe.plot('Author', 'Score', kind='bar')
plt.show()

print("CHART: AUTHOR / AUTHOR STEADINESS")
dataframe = pd.DataFrame({'Author': authors, 'Steadiness': author_steadiness})
dataframe.plot('Author', 'Steadiness', kind='bar')
plt.show()

PAPER SCORE:        [0.80 0.50]
PAPER STEADINESS:   [1.00 0.55]
READER SCORE:       [1.00 0.23 0.50 1.00]
READER STEADINESS:  [1.00 1.00 1.55 1.55]
AUTHOR SCORE:       [0.69]
AUTHOR STEADINESS:  [1.55]
CHART: PAPER / PAPER SCORE
CHART: PAPER / PAPER STEADINESS
CHART: READER / READER SCORE


CHART: READER / READER STEADINESS


CHART: AUTHOR / AUTHOR SCORE
CHART: AUTHOR / AUTHOR STEADINESS
